<a href="https://colab.research.google.com/github/yanncoadou/MLtutorials/blob/main/ED352_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>IDPASC 2021 Machine learning hands-on</h1>

# Standard imports and practical functions

In [ ]:
# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_circles
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, accuracy_score, roc_auc_score, roc_curve, RocCurveDisplay

%matplotlib inline
import seaborn as sns # seaborn for nice plots
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
np.random.seed(31415) # set the np random seed for reproducibility

### Function to plot decision contours

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def my_plot_decision_regions(model, X, y, alpha=1.0, size=25, npts=10000, zoom=0.25, event5=False):
  x1min = X[:,0].min() - zoom
  x1max = X[:,0].max() + zoom

  x2min = X[:,1].min() - zoom
  x2max = X[:,1].max() + zoom
  
  x1 = np.random.uniform(x1min, x1max, npts)
  x2 = np.random.uniform(x2min, x2max, npts)

  if hasattr(model, "predict_proba"):
    z = model.predict_proba(np.vstack((x1,x2)).T)
  else:
    z = model.predict(np.vstack((x1,x2)).T)
  
  if len(z.shape) == 2:
    if z.shape[1] == 1:
      z = z.reshape(-1)
    elif z.shape[1] == 2:
      z = z[:,1].reshape(-1)

  fig, ax = plt.subplots()

  bottom = cm.get_cmap('Oranges', 128)
  top = cm.get_cmap('Blues_r', 128)

  newcolors = np.vstack((top(np.linspace(0, 1, 128+128)[-128:]),
                        bottom(np.linspace(0, 1, 128+128)[:128])))
  newcmp = ListedColormap(newcolors, name='OrangeBlue')


  ax.tricontour(x1, x2, z, levels=np.linspace(0.0-np.finfo(float).eps,1.0+np.finfo(float).eps,20,True), linewidths=0.1, colors='k', antialiased=True)
  cntr = ax.tricontourf(x1, x2, z, levels=np.linspace(0.0-np.finfo(float).eps,1.0+np.finfo(float).eps,20,True), cmap=newcmp)
  sctr0 = ax.scatter(X[y==0][:,0], X[y==0][:,1], alpha=alpha, s=size, c="#1f77b4", marker="s", edgecolors="k", linewidths=0.5)
  sctr1 = ax.scatter(X[y==1][:,0], X[y==1][:,1], alpha=alpha, s=size, c="#ff7f0e",  marker="^", edgecolors="k", linewidths=0.5)
  if event5: # showing particular swinger event
    sctr2 = ax.scatter(X[4][0], X[4][1], alpha=1, s=size*10, c="lightgreen",  marker="X", edgecolors="k", linewidths=1)
  fig.colorbar(cntr, ax=ax)
  # ax.set(xlim=(x1min, x1max), ylim=(x2min, x2max))

  plt.show()

### Function to plot ROC curve

In [ ]:
def my_plot_roc_curve(model, X_test, y_test):
  if hasattr(model, "predict_proba"):
    y_scores = model.predict_proba(X_test)
  else:
    y_scores = model.predict(X_test)

  if len(y_scores.shape) == 2:
    if y_scores.shape[1] == 1:
      y_scores = y_scores.reshape(-1)
    elif y_scores.shape[1] == 2:
      y_scores = y_scores[:,1].reshape(-1)
  fpr, tpr, _ = roc_curve(y_test, y_scores)
  roc_auc = roc_auc_score(y_test, y_scores)
  plt.clf()
  display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=model.__class__.__name__)
  display.plot()
  plt.plot([0, 1], [0, 1], color='black', linestyle='--')
  plt.show()

# Classifier zoo

In [ ]:
# X = (x,y) coordinates; y = class
X1, y1 = make_circles(n_samples=1000, noise=0.1, factor=0.8)
X2, y2 = make_circles(n_samples=1000, noise=0.2, factor=0.2)
X = np.vstack((X1,X2/2))
y = np.hstack((y1,y2))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y);

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(15,10))
plot_tree(dtc)
plt.show();


In [ ]:
accuracy_score(y_test, dtc.predict(X_test))

Access to results:
- `predict` returns the class (0 or 1 if binary classifier)
- `predict_proba` returns the probability of each class



In [ ]:
print("predict: \n",dtc.predict(X_test[:5]))
print("predict_proba: \n",dtc.predict_proba(X_test[:5]))

In [ ]:
try:
  from mlxtend.plotting import plot_decision_regions
except ImportError as e:
  !pip install mlxtend
  from mlxtend.plotting import plot_decision_regions

In [ ]:
# practical but limited contour-plotting function
plot_decision_regions(X_test, y_test, dtc);

In [ ]:
# defined at top of notebook
# can use class (0 or 1) or class probability when available
my_plot_decision_regions(dtc, X_test, y_test)

In [ ]:
my_plot_roc_curve(dtc, X_test, y_test)

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
#abc = AdaBoostClassifier()
abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),n_estimators=100)
abc.fit(X_train, y_train)

In [ ]:
print("predict: \n",abc.predict(X_test[:5]))
print("predict_proba: \n",abc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(abc, X_test, y_test)
my_plot_roc_curve(abc, X_test, y_test)

### Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=400)
gbc.fit(X_train, y_train)

In [ ]:
print("predict: \n",gbc.predict(X_test[:5]))
print("predict_proba: \n",gbc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(gbc, X_test, y_test, event5=True)
my_plot_roc_curve(gbc, X_test, y_test)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=400)
rfc.fit(X_train, y_train)

In [ ]:
print("predict: \n",rfc.predict(X_test[:5]))
print("predict_proba: \n",rfc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(rfc, X_test, y_test)
my_plot_roc_curve(rfc, X_test, y_test)

### Comparison

In [ ]:
y_preds_dtc = dtc.predict_proba(X_test)[:,1].reshape(-1)
y_preds_abc = abc.predict_proba(X_test)[:,1].reshape(-1)
y_preds_gbc = gbc.predict_proba(X_test)[:,1].reshape(-1)
y_preds_rfc = rfc.predict_proba(X_test)[:,1].reshape(-1)
fpr_dtc,tpr_dtc,_ = roc_curve(y_true=y_test, y_score=y_preds_dtc)
fpr_abc,tpr_abc,_ = roc_curve(y_true=y_test, y_score=y_preds_abc)
fpr_gbc,tpr_gbc,_ = roc_curve(y_true=y_test, y_score=y_preds_gbc)
fpr_rfc,tpr_rfc,_ = roc_curve(y_true=y_test, y_score=y_preds_rfc)
auc_test_dtc = roc_auc_score(y_true=y_test, y_score=y_preds_dtc)
auc_test_abc = roc_auc_score(y_true=y_test, y_score=y_preds_abc)
auc_test_gbc = roc_auc_score(y_true=y_test, y_score=y_preds_gbc)
auc_test_rfc = roc_auc_score(y_true=y_test, y_score=y_preds_rfc)
plt.plot(fpr_dtc, tpr_dtc, color='darkblue',label='{} (AUC  = {})'.format(dtc.__class__.__name__,np.round(auc_test_dtc,decimals=2)))
plt.plot(fpr_abc, tpr_abc, color='darkred',label='{} (AUC  = {})'.format(abc.__class__.__name__,np.round(auc_test_abc,decimals=2)))
plt.plot(fpr_gbc, tpr_gbc, color='darkgreen',label='{} (AUC  = {})'.format(gbc.__class__.__name__,np.round(auc_test_gbc,decimals=2)))
plt.plot(fpr_rfc, tpr_rfc, color='darkorange',label='{} (AUC  = {})'.format(rfc.__class__.__name__,np.round(auc_test_rfc,decimals=2)))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right");


### Neural networks
It is possible to use neural networks from scikit-learn (`from sklearn.neural_network import MLPClassifier`). In the following we will use more advanced implementations with [Tensorflow](https://www.tensorflow.org).

In [ ]:
try:
  import tensorflow as tf
except ImportError as e:
  !pip install tensorflow
  import tensorflow as tf
print (tf.__version__)  # preinstalled version 2.6.0 20210824
from tensorflow import keras

In [ ]:
model = keras.models.Sequential(
    [
     keras.layers.Dense(128, activation='relu', input_shape=(2,)),
     keras.layers.Dense(128, activation='relu'),
     keras.layers.Dense(1, activation='sigmoid')
    ]
)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', keras.metrics.AUC()])
model.summary()

In [ ]:
model.fit(X_train, y_train,
          validation_split=0.2,
          epochs=1000,
          callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

In [ ]:
print("predict: \n",model.predict(X_test[:5]))
accuracy_score(y_test, model.predict(X_test).round())

In [ ]:
my_plot_decision_regions(model, X_test, y_test)
my_plot_roc_curve(model, X_test, y_test)

In [ ]:
y_preds_model = model.predict(X_test)
fpr_model,tpr_model,_ = roc_curve(y_true=y_test, y_score=y_preds_model)
auc_test_model = roc_auc_score(y_true=y_test, y_score=y_preds_model)
plt.plot(fpr_rfc, tpr_rfc, color='darkorange',label='{} (AUC  = {})'.format(rfc.__class__.__name__,np.round(auc_test_rfc,decimals=2)))
plt.plot(fpr_model, tpr_model, color='purple',label='{} (AUC  = {})'.format("Neural network",np.round(auc_test_model,decimals=2)))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right");


# High energy physics application

## Input dataset

Data created from ATLAS Open Data by David Rousseau. See doc:

http://opendata.atlas.cern/release/2020/documentation/datasets/intro.html

### Downloading dataset

In [ ]:
import os
filename="dataWW_d1_600k.csv.gz"
if not os.path.isfile(filename):
  try:
    import gdown
  except ImportError as e:
    !pip install gdown
    import gdown
  !gdown https://drive.google.com/uc?id=1nlXp7P-xq_jip4aPE0j0mnPhYnIOcBv4
!ls -lrt

# Loading dataset
dfall = pd.read_csv(filename) 
print ("\nFile loaded with ",dfall.shape[0], " events ")


After downloading message and directory listing, you should now see:

`File loaded with  600000  events`

### Checking the content

In [ ]:
#dumping list of features
dfall.columns

In [ ]:
#examining first few events
display(dfall.head())

In [ ]:
#examining feature distributions
dfall.describe()

***Event weights***

In [ ]:
label_nevents = (dfall[dfall.label==0].shape[0], dfall[dfall.label==1].shape[0] )
print("Number of events per class (B, S):",label_nevents)

label_weights = (dfall[dfall.label==0].mcWeight.sum(), dfall[dfall.label==1].mcWeight.sum() ) 
print("Total weight per class (B, S):    ",label_weights)

## Event selection

Only keep events with exactly two leptons for this exercise.

Only keep events with positive weight, as many ML tools choke on negative weight.

*Note: This is in principle WRONG, only valid if your positive and negative weight events are statistically similar (could then also take the absolute value of the weight to increase statistics).*


In [ ]:
print ("Df shape before selection:", dfall.shape)

fulldata=dfall[ (dfall.lep_n==2) & (dfall.mcWeight > 0)]  

print ("Df shape after selection: ",fulldata.shape)

In [ ]:
# Hide label and weights in separate vectors (not discriminating features)
# WARNING : there should be neither selection nor shuffling later on! (otherwise misalignement)
target = fulldata["label"]
weights = fulldata["mcWeight"]

# for simplicity only keep some features
# this is actually making a deep copy from fulldata
data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_phi_0', 'lep_phi_1'])
#data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_eta_0', 'lep_eta_1', 'lep_phi_0', 'lep_phi_1','jet_n','jet_pt_0',
#       'jet_pt_1', 'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1']

print ("Df shape of dataset to be used:",data.shape)

### Event weights

In [ ]:
fig,ax=plt.subplots()

bins=np.linspace(-1,1,101)
plt.hist(weights[target==0]*1000,bins=bins,color='b',alpha=0.5,density=True,label='B ackground')
plt.hist(weights[target==1]*10000,bins=bins,color='r',alpha=0.5,density=True,label='S ignal (Wx10)')
plt.legend(loc='best')
ax.set_xlabel('weight*1000')
plt.show()

### Feature engineering

Add more complex variables to the dataset.

*Do this later if time permits.*

In [ ]:
if False: 
    data["lep_deltaphi"]=np.abs(np.mod(data.lep_phi_1-data.lep_phi_0+3*np.pi,2*np.pi)-np.pi)

    print (data.shape)
    display(data.head())



### Plotting variables

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))
data['met_et'].plot.hist(title='Missing Transverse Energy', log=True, ax=ax[0])
data[data.lep_pt_0+data.lep_pt_1>1000]['met_et'].plot.hist(bins=np.linspace(0,400,50),title='Missing Transverse Energy for large lepton Pt', ax=ax[1]);

In [ ]:
ax=data[target==0].plot.scatter(x='met_et', y='lep_pt_0',color="b",label="B")
data[target==1].plot.scatter(x='met_et', y='lep_pt_0',color="r",label="S",alpha=.5,ax=ax);

In [ ]:
data[data.lep_pt_0+data.lep_pt_1>2000].head()

In [ ]:
ax=data[target==0].hist(weights=weights[target==0],figsize=(15,12),bins=50,color='b',alpha=0.5,density=True,label="B")
ax=ax.flatten()[:data.shape[1]] # to avoid error if holes in the grid of plots (like if 7 or 8 features)
data[target==1].hist(weights=weights[target==1],figsize=(15,12),bins=50,color='r',alpha=0.5,density=True,ax=ax,label="S");


### Features correlation matrix

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))

corrMatrix = data[target==0].corr()
ax[0].set_title("Background features correlation matrix")
sns.heatmap(corrMatrix.round(3), ax=ax[0], annot=True);

corrMatrix = data[target==1].corr()
ax[1].set_title("Signal features correlation matrix")
sns.heatmap(corrMatrix.round(3), ax=ax[1], annot=True);


## Sample splitting

In [ ]:
np.random.seed(31415) # set the random seed (used for the train/test splitting)

from sklearn.model_selection import train_test_split
train_size = 0.75 # fraction of sample used for training
val_size = 0.2 # fraction of training sample used for validation

# split only train/test
#X_train, X_test, y_train, y_test, weights_train, weights_test = \
#    train_test_split(data, target, weights, train_size=train_size)

#split in train/validation/test
X_holdout, X_test, y_holdout, y_test, weights_holdout, weights_test = \
    train_test_split(data, target, weights, train_size=train_size)
X_train, X_val, y_train, y_val, weights_train, weights_val = \
    train_test_split(X_holdout, y_holdout, weights_holdout, train_size=1-val_size)

print("Training sample:  ", X_train.shape)
print("Validation sample:", X_val.shape)
print("Testing sample:   ", X_test.shape)

class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())
print ("class_weights_train (B, S):",class_weights_train)

for i in range(len(class_weights_train)):
    weights_train[y_train == i] *= max(class_weights_train)/ class_weights_train[i] #equalize number of background and signal event
    weights_test[y_test == i] *= 1/(1-train_size) # increase test weight to compensate for sampling
    weights_val[y_val == i] *= 1/val_size/train_size # increase val weight to compensate for samplings
    
print ("Test:  total weight sig", weights_test[y_test == 1].sum())
print ("Test:  total weight bkg", weights_test[y_test == 0].sum())
print ("Train: total weight sig", weights_train[y_train == 1].sum())
print ("Train: total weight bkg", weights_train[y_train == 0].sum())
print ("Val:   total weight sig", weights_val[y_val == 1].sum())
print ("Val:   total weight bkg", weights_val[y_val == 0].sum())



## Network training

In [ ]:
try:
  import tensorflow as tf
except ImportError as e:
  !pip install tensorflow
  import tensorflow as tf
print (tf.__version__)  # preinstalled version 2.6.0 20210824
from tensorflow import keras

In [ ]:
tf.random.set_seed(1234) # to have reproducible networks
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), # 1st hidden layer
  #tf.keras.layers.Dense(128, activation='relu'), # 2nd hidden layer
  tf.keras.layers.Dense(1,activation="sigmoid") # output layer
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
              weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])

history = model.fit(X_train, y_train.values,
                    epochs=1,
                    #validation_split=0.2,   # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val),
                    batch_size=1024,
                    sample_weight=weights_train.values,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

### Standardisation of inputs


In [ ]:
from sklearn.preprocessing import StandardScaler

print("Original mean and variance:")
for feature, mean, std in zip(data.columns,X_train.mean(0), X_train.std(0)):
  print("{:9}: {:7.4f} +/- {:7.4f}".format(feature,mean,std))

# Standardize features by removing the mean and scaling to unit variance
# in training sample
scaler = StandardScaler()
# ".values[:]" to keep dataframe and not convert to numpy array
X_train.values[:] = scaler.fit_transform(X_train)
# apply to testing/validation sample the transformation calculated on training sample
X_test.values[:] = scaler.transform(X_test)
X_val.values[:] = scaler.transform(X_val)

print("\nStandardised mean and variance:")
for feature, mean, std in zip(data.columns,X_train.mean(0), X_train.std(0)):
  print("{:9}: {:7.4f} +/- {:7.4f}".format(feature,mean,std))

In [ ]:
tf.random.set_seed(1234) # to have reproducible networks
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), # 1st hidden layer
  #tf.keras.layers.Dense(128, activation='relu'), # 2nd hidden layer
  tf.keras.layers.Dense(1,activation="sigmoid") # output layer
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
              weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])

history = model.fit(X_train, y_train.values,
                    epochs=100,
                    #validation_split=0.2,   # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val),
                    batch_size=1024,
                    sample_weight=weights_train.values,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

y_pred_model = model.predict(X_test).reshape(-1)

*Compare the training loss/accuracy/AUC after epoch 1 with that obtained before standardisation.*

In [ ]:
density=True   # normalised to 1 (=> probability density function)
#density=False   # normalised to one year at LHC

plt.hist(y_pred_model[y_test == 0],
         color='b', alpha=0.5, 
         bins=30,
         histtype='stepfilled', density=density,
         label='B (test)', weights=weights_test[y_test == 0])
plt.hist(y_pred_model[y_test == 1],
         color='r', alpha=0.5,
         bins=30,
         histtype='stepfilled', density=density,
         label='S (test)', weights=weights_test[y_test == 1])
plt.legend()
plt.title("NN model score");

In [ ]:
my_plot_roc_curve(model, X_test, y_test)

### Training monitoring

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(history.history['loss'],label="Training loss")
ax[0].plot(history.history['val_loss'],label="Validation loss")
ax[0].set_xlabel("Epoch")
ax[0].legend(loc='best');

ax[1].plot(history.history['auc'],label="Training AUC")
ax[1].plot(history.history['val_auc'],label="Validation AUC")
ax[1].set_xlabel("Epoch")
ax[1].legend(loc='best');

### Model saving

***Whole-model saving & loading***

You can save an entire model to a directory. It will include:
- the model's architecture/config
- the model's weight values (which were learned during training)
- the model's compilation information (if `compile()` was called)
- the optimizer and its state, if any (this enables you to restart training where you left)


In [ ]:
model.save("NNmodel")
!ls -a NNmodel/*

In [ ]:
print("Prediction from original model:")
display(model.predict(X_test[:5]))

reloaded_model=keras.models.load_model("NNmodel")
print("Prediction from reloaded model:")
display(reloaded_model.predict(X_test[:5]))
#np.testing.assert_allclose(
#    model.predict(X_test), reloaded_model.predict(X_test)
#)

# further training
reloaded_model.fit(X_train, y_train.values,
                   epochs=5,
                   #validation_split=0.2,   # to be used with train/test split
                   validation_data=(X_val, y_val, weights_val),
                   batch_size=1024,
                   sample_weight=weights_train.values,
                   callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

*Compare first epoch values with original training.*

***Partial save***

Save a single HDF5 file containing the model's architecture, weights values, and `compile()` information.

Not saved (to be provided separately to resume training):
- external losses & metrics added via `model.add_loss()` & `model.add_metric()`
- computation graph of custom objects

In [ ]:
model.save("NNmodel.h5")
!ls -lrt --color
print("\nPrediction from original model:")
display(model.predict(X_test[:5]))

reloaded_model=keras.models.load_model("NNmodel.h5")
print("Prediction from reloaded model:")
display(reloaded_model.predict(X_test[:5]))

***Saving the architecture and weights***

Keeping the model's configuration and training weights in separate files

In [ ]:
arch = model.to_json()
with open('NNmodel.json', 'w') as arch_file:
  arch_file.write(arch)
model.save_weights('NNmodel_weights.h5')
!ls -lrt --color

In [ ]:
!python -m json.tool NNmodel.json

In [ ]:
with open('NNmodel.json', 'r') as f:
  reloaded_model = keras.models.model_from_json(f.read())
reloaded_model.summary()

reloaded_model.load_weights("NNmodel_weights.h5")
#reloaded_model.compile(loss="binary_crossentropy",
#                       optimizer="adam",
#                       #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
#                       weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])
reloaded_model.fit(X_train, y_train.values,
                    epochs=1,
                    #validation_split=0.2,   # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val),
                    batch_size=1024,
                    sample_weight=weights_train.values,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])


## Physics performance

### Significance

Asimov significance (from [arXiv:1007.1727](https://arxiv.org/abs/1007.1727) eq. 97):

> AMS = $\sqrt{2\left((s+b)\ln\left(1+\frac{s}{b}\right) - s\right)} = \frac{s}{\sqrt{b}}\left(1+\mathcal{O}(s/b)\right)$

In [ ]:
from math import sqrt
from math import log
def amsasimov(s,b):
  if b<=0 or s<=0:
      return 0
  try:
      return sqrt(2*((s+b)*log(1+float(s)/b)-s))
  except ValueError:
      print(1+float(s)/b)
      print (2*((s+b)*log(1+float(s)/b)-s))

In [ ]:
int_pred_test_sig_model = [weights_test[(y_test ==1) & (y_pred_model > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
int_pred_test_bkg_model = [weights_test[(y_test ==0) & (y_pred_model > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]

vamsasimov_model = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig_model,int_pred_test_bkg_model)]
print("Z: ",np.round(max(vamsasimov_model),decimals=3))


In [ ]:
plt.plot(np.linspace(0,1,num=50),vamsasimov_model, label='AMS (Z_max = {})'.format(np.round(max(vamsasimov_model),decimals=3)))

plt.title("Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
#plt.savefig("Significance.pdf")
plt.show()

### Feature importance
Feature importance allows to display the importance of each feature without rerunnning the training. It is obtained from internal algorithm quantities, like cumulated decrease of impurity. Magnitude is arbitrary. It can be used as a not very reliable indication of which feature is the most discriminant.

Very straightforward with decision trees.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=10,verbose=1)
gbc.fit(X_train, y_train, sample_weight=weights_train)

In [ ]:
plt.bar(data.columns.values, gbc.feature_importances_)
plt.xticks(rotation=90)
plt.title("Feature importance")
plt.show()

*What about a different tree classifier?*

In [ ]:
# preinstalled version 0.9.0 20210824
!pip install xgboost --upgrade # install 1.4.2 20210824
import xgboost as xgb
print(xgb.__version__)

useLGB=False #Could also use LightGBM
# preinstalled version 2.2.3 20210824
if useLGB:
  !pip install lightgbm --upgrade # install 3.2.1 20210824
  import lightgbm as lgb
  print (lgb.__version__)

In [ ]:
from xgboost import XGBClassifier
# tree_method="hist" is 10 times faster, however less robust against awkwards features (not a bad idea to double check without it)
# can even try tree_method="gpu_hist" if proper GPU installation
# use_label_encoder and eval_metric to silence warning in 1.3.0
xgb = XGBClassifier(tree_method="hist",use_label_encoder=False,eval_metric='logloss')

xgb.fit(X_train, y_train.values, sample_weight=weights_train.values) # note that XGB 1.3.X requires positive weight



In [ ]:
if useLGB:
  gbm = lgb.LGBMClassifier()
  gbm.fit(X_train, y_train.values,sample_weight=weights_train.values)


In [ ]:
if useLGB:
  fig,ax=plt.subplots(1, 3, figsize=(18, 5))
else:
  fig,ax=plt.subplots(1, 2, figsize=(12, 5))
ax[0].bar(data.columns.values, xgb.feature_importances_)
ax[0].tick_params(labelrotation=90)
ax[0].set_title("XGBoost feature importance")
ax[1].bar(data.columns.values, gbc.feature_importances_)
ax[1].tick_params(labelrotation=90)
ax[1].set_title("sklearn feature importance");
if useLGB:
  ax[2].bar(data.columns.values, gbm.feature_importances_)
  ax[2].tick_params(labelrotation=90)
  ax[2].set_title("LightGBM feature importance");


### Permutation importance

A better way to show the importance of each feature is Permutation Importance, where each feature in turn is replaced by an instance of an other event (effectively switching it off by randomising).

Works on any classifier, not just DT-based.

However, report can be misleading in case of highly correlated variables.

Available in [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html) but without event weights in Colab version (0.22).
   


In [ ]:
if False:
  from sklearn.inspection import permutation_importance
  result_xgb = permutation_importance(xgb, X_test, y_test, n_repeats=1, random_state=42, n_jobs=2)
  forest_importances_xgb = pd.Series(result_xgb.importances_mean, index=list(data.columns.values))

  if useLGB:
    result_gbm = permutation_importance(gbm, X_test, y_test, n_repeats=1, random_state=42, n_jobs=2)
    forest_importances_gbm = pd.Series(result_gbm.importances_mean, index=list(data.columns.values))

  if useLGB:
    fig,ax=plt.subplots(1, 2, figsize=(12, 5))
    forest_importances_xgb.plot.bar(ax = ax[0], subplots=True)
    ax[0].set_title("XGBoost permutation importance")
  else:
    fig,ax=plt.subplots()
    forest_importances_xgb.plot.bar()
    ax.set_title("XGBoost permutation importance")
  if useLGB:
    forest_importances_gbm.plot.bar(ax = ax[1], subplots=True)
    ax[1].set_title("LightGBM permutation importance")


Another implementation targetting HEP:

https://github.com/aghoshpub/permutationImportancePhysics 

In particular it allows to : 
   * use event weights
   * display directly the loss in whatever criterion (ROC auc, asimov significance) when the feature is switched off
   * display the feature importance for a specific subset (for example the most signal like)
   * it can even display which feature has the largest impact on systematics


In [ ]:
if False:
    !pip install PermutationImportancePhysics
    from permutationimportancephysics.PermutationImportance import PermulationImportance # note the delibrate typo PermuLation
    #XGBoost
    PI_xgb = PermulationImportance(model=xgb, X=X_test.values,y=y_test,weights=weights_test,\
                           n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
    #PI_xgb.dislayResults()
    plott_xgb = PI_xgb.plotBars()

    #LightGBM    
    if useLGB:
      PI_gbm = PermulationImportance(model=gbm, X=X_test.values,y=y_test,weights=weights_test,\
                             n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
      #PI_gbm.dislayResults()
      plott_gbm = PI_gbm.plotBars()

    # also works with Keras NN
    PI_model = PermulationImportance(model=model, X=X_test.values,y=y_test,weights=weights_test,\
                           n_iterations=1,usePredict_poba=False, scoreFunction="amsasimov", colNames=list(data.columns.values))
    #PI_model.dislayResults()
    plott_model = PI_model.plotBars()
    


### Hyperparameter optimisation
Can be done by hand, with [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) or [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Also dedicated packages doing Gaussian process optimisation or 'tree of Parzen estimators' (TPE) (e.g. [hyperopt](https://github.com/hyperopt/hyperop) or [optuna](https://optuna.org/)).

In [ ]:
import scipy.stats as stats
if False:
    from sklearn.model_selection import RandomizedSearchCV

    # specify parameters and distributions to sample from
    param_dist_XGB = {'n_estimators': stats.randint(50, 500), #default 100
                      'learning_rate': stats.uniform(0.1, 0.5)} #def 0.3 
                      #'max_depth': stats.randint(3, 12)} # default 6


    # default CV is 5 fold, reduce to 2 for speed concern
    # default n_iter is 10 sets of parameters, reduce to 5 for speed concern
    gsearch = RandomizedSearchCV(estimator = XGBClassifier(tree_method="hist",use_label_encoder=False,eval_metric='logloss'), 
                        param_distributions = param_dist_XGB, 
                        scoring='roc_auc',n_iter=5,cv=2,verbose=2)
    gsearch.fit(X_train,y_train, sample_weight=weights_train)

    print ("Best parameters: ",gsearch.best_params_)
    print ("Best score (on train dataset CV): ",gsearch.best_score_)


    y_pred_gs = gsearch.predict_proba(X_test)[:,1]
    print("... corresponding score on test dataset: ",roc_auc_score(y_true=y_test, y_score=y_pred_gs, sample_weight=weights_test))
    dfsearch=pd.DataFrame.from_dict(gsearch.cv_results_)
    display(dfsearch)
    dfsearch.plot.scatter("param_n_estimators","mean_test_score")
    #dfsearch.plot.scatter("param_max_depth","mean_test_score")
    dfsearch.plot.scatter("param_learning_rate","mean_test_score")